In [37]:
import requests
from bs4 import BeautifulSoup
import csv

In [ ]:
link_structure = 'https://fbref.com/es/jugadores/{id_player}/partidos/2023-2024/{type_stats}/'

In [50]:
id_player = '3ee0dd59'
player_link = f'https://fbref.com/es/jugadores/{id_player}/partidos/2023-2024/'
response = requests.get(player_link)

# Ver contenido de la página 
soup = BeautifulSoup(response.content, 'html.parser')

tabla = soup.find('table', {'id': 'matchlogs_all'})

# Obtener filas de la tabla
datos_tabla = []

# Extrae los encabezados de la tabla
headers = [th.get('aria-label') for th in tabla.find('thead').find_all('tr')[1].find_all('th')]

# Extrae las filas del cuerpo de la tabla
filas = tabla.select('tbody tr:not([class])')

for fila in filas:
    # Extrae las celdas de cada fila
    celdas = fila.find_all(['th', 'td'])
    datos_fila = [celda.text.strip() for celda in celdas]
    datos_tabla.append(datos_fila)

# Imprime los datos de la tabla
print("Encabezados:", headers)
for fila in datos_tabla:
    print(fila)


Encabezados: ['Fecha', 'Día', 'Competencia', 'Ronda', 'Sedes', 'Resultado', 'Equipo', 'Adversario', 'Arranque', 'Posición', 'Minutos', 'Goles', 'Asistencias', 'Tiros penales ejecutados', 'Tiros penales intentados', 'Total de disparos', 'Lanzamientos en el Objetivo', 'Tarjetas amarillas', 'Tarjetas rojas', 'Toques', 'Derribos', 'Intercepciones', 'Bloqueos', 'xG: Goles esperados', 'npxG: Goles esperados (xG) sin contar penaltis', 'xAG: Exp. Objetivos asistidos', 'Acciones para la creación de tiros', 'Acciones para la creación de goles', 'Pases completados', 'Pases intentados', '% de pase completo', 'Pases progresivos', 'Transportes', 'Acarreos progresivos', 'Tomas intentadas', 'Tomas exitosas', 'Informe del partido']
['2023-08-12', 'Sáb', 'La Liga', 'Semana 1 de partido', 'Local', 'E 1–1', 'Real Sociedad', 'Girona', 'Sí', 'CM', '90', '0', '0', '0', '0', '1', '0', '0', '0', '52', '1', '0', '1', '0.0', '0.0', '0.0', '4', '0', '42', '47', '89.4', '5', '34', '2', '2', '1', 'Informe del parti

Función para extraer datos de los partidos de un jugador en una temporada. Faltaría cómo obtener los ids y posibilidad de incluir otras tablas de otros tipos de estadísticas.

In [49]:
def get_player_csv(id_player):
    player_link = f'https://fbref.com/es/jugadores/{id_player}/partidos/2023-2024/'
    response = requests.get(player_link)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        tabla = soup.find('table', {'id': 'matchlogs_all'}) # Se queda con la tabla de partidos
        datos_tabla = []
        
        # Extraemos los encabezados de la tabla
        fila_encabezados = tabla.find('thead').find_all('tr')[1] # Nos quedamos con la segunda fila de encabezados
        headers = [th.get('aria-label') for th in fila_encabezados.find_all('th')] 
        
        # Extraemos las filas de la tabla
        filas = tabla.select('tbody tr:not([class])') # Evitamos filas separadoras y de partidos que no ha jugado
        for fila in filas:
            celdas = fila.find_all(['th', 'td'])
            datos_fila = [celda.text.strip() for celda in celdas]
            datos_tabla.append(datos_fila)

        # Guardamos los datos en un archivo CSV
        with open(f'{id_player}_data.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(headers)
            writer.writerows(datos_tabla)

In [44]:
get_player_csv('3ee0dd59')